In [1]:
!pip uninstall datasets -y
!pip install datasets


Found existing installation: datasets 2.14.4
Uninstalling datasets-2.14.4:
  Successfully uninstalled datasets-2.14.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu

In [2]:
# Install dependencies
!pip install transformers datasets gradio -q

# Imports
from transformers import BartForConditionalGeneration, BartTokenizerFast, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset
import gradio as gr
import torch

# Load pretrained model & tokenizer
model_name = "facebook/bart-base"
tokenizer = BartTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Load dataset (demo subset)
data = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="train[:1%]")

# Preprocessing
def preprocess(example):
    inputs = tokenizer(
        example["article"], max_length=512, truncation=True, padding="max_length", return_tensors="pt"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["highlights"], max_length=128, truncation=True, padding="max_length"
        )
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply preprocessing
tokenized_data = data.map(preprocess, batched=True, remove_columns=data.column_names)
tokenized_data.set_format(type="torch")

# Training arguments with early stopping
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,  # Set higher and let early stopping handle when to stop
    logging_steps=3,
    eval_strategy="steps", # Changed from evaluation_strategy to eval_strategy
    eval_steps=500,
    save_steps=500,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# Define Trainer with EarlyStopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data,  # You can split the train into train/validation if needed
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# Train
trainer.train()

# Save model
model.save_pretrained("./bart-legal-summarizer")
tokenizer.save_pretrained("./bart-legal-summarizer")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss
500,1.404700,0.891007
1000,0.860600,0.757338
1500,0.723800,0.664556
2000,0.948000,0.629077


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


('./bart-legal-summarizer/tokenizer_config.json',
 './bart-legal-summarizer/special_tokens_map.json',
 './bart-legal-summarizer/vocab.json',
 './bart-legal-summarizer/merges.txt',
 './bart-legal-summarizer/added_tokens.json',
 './bart-legal-summarizer/tokenizer.json')

In [5]:
!pip install gradio -q
!pip install PyPDF2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.0 MB/s eta 0:00:00


In [6]:
from transformers import pipeline
import gradio as gr
import PyPDF2

# Load the fine-tuned model (on CPU)
summarizer = pipeline(
    "summarization",
    model="./bart-legal-summarizer",
    tokenizer="./bart-legal-summarizer",
    device=-1  # Use CPU
)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_file):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text
    return text

# Gradio function
def summarize_legal_doc(text_input, pdf_file):
    # Prefer PDF if provided
    if pdf_file is not None:
        text = extract_text_from_pdf(pdf_file)
    elif text_input.strip() != "":
        text = text_input
    else:
        return "Please provide either text or a PDF document."

    # Truncate text to 1024 tokens (~3000-4000 chars) if too long for BART
    if len(text) > 3500:
        text = text[:3500]

    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Gradio Interface
gr.Interface(
    fn=summarize_legal_doc,
    inputs=[
        gr.Textbox(lines=10, label="Paste Legal Text Here"),
        gr.File(label="Or Upload PDF", file_types=[".pdf"])
    ],
    outputs=gr.Textbox(label="Summary"),
    title="Legal Document Summarizer (BART)",
    description="Paste a legal document or upload a PDF to get a concise summary."
).launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://87036444b55af75b53.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
!pip install rouge_score
!pip install rouge_score datasets -q



  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f391956b4d5a054e5bb58c5d609f527bc8b0f6fb203d1d989a4debab23f28c00
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [8]:
from transformers import pipeline
from datasets import load_dataset
from rouge_score import rouge_scorer
import pandas as pd
from tqdm import tqdm

# Load the test split of CNN/DailyMail (or your own dataset here)
dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="test[:1%]")  # Use more for real eval

# Load your fine-tuned model
summarizer = pipeline(
    "summarization",
    model="./bart-legal-summarizer",
    tokenizer="./bart-legal-summarizer",
    device=-1  # CPU; use 0 for GPU
)

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Evaluate
results = []
for sample in tqdm(dataset):
    input_text = sample['article']
    reference_summary = sample['highlights']

    # Truncate input for BART if too long
    if len(input_text) > 3500:
        input_text = input_text[:3500]

    generated_summary = summarizer(input_text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    scores = scorer.score(reference_summary, generated_summary)

    results.append({
        "rouge1": scores['rouge1'].fmeasure,
        "rouge2": scores['rouge2'].fmeasure,
        "rougeL": scores['rougeL'].fmeasure,
        "reference": reference_summary,
        "generated": generated_summary
    })

# Convert to DataFrame
df = pd.DataFrame(results)

# Display ROUGE scores
print("\nAverage ROUGE Scores:")
print(df[["rouge1", "rouge2", "rougeL"]].mean())

# Optionally, save results to CSV
df.to_csv("rouge_eval_results.csv", index=False)


Device set to use cpu
100%|██████████| 115/115 [15:36<00:00,  8.14s/it]


Average ROUGE Scores:
rouge1    0.331414
rouge2    0.129726
rougeL    0.232880
dtype: float64


In [ ]:
import os

print(os.listdir("./results"))
